# IMDB Random Forests Regression
![ImdbIcon](../images/imdbheader.jpg)

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, plot_confusion_matrix, classification_report

In [4]:
final_df = pd.read_csv('../data/final_df.csv')

In [5]:
final_df.columns

Index(['movie_title', 'year', 'tagline', 'plot', 'cast', 'duration', 'Action',
       'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime', 'Drama',
       'Family', 'Fantasy', 'History', 'Horror', 'Music', 'Musical', 'Mystery',
       'Romance', 'Sci-Fi', 'Sport', 'Thriller', 'War', 'Western', 'avg_vote',
       'total_votes', 'us_voters_votes', 'votes', 'votes_1', 'votes_2',
       'votes_3', 'votes_4', 'votes_5', 'votes_6', 'votes_7', 'votes_8',
       'votes_9', 'votes_10', 'popularity', 'director_score', 'actor_score',
       'actress_score', 'tagline_sentiment', 'plot_sentiment', 'total_score',
       'profitable', 'budget', 'revenue', 'budget_adj', 'revenue_adj'],
      dtype='object')

## Random Forests Regression

In [6]:
features = ['Action', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime', 'Drama', 'Family', 'Fantasy', 'History', 
            'Horror', 'Music', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Sport', 'Thriller', 'War', 'Western', 
            'us_voters_votes', 'total_votes', 'votes_4', 'votes_5', 'votes_6', 'votes_7', 'votes_8','votes_9', 'votes_10', 
            'popularity', 'director_score', 'actor_score', 'actress_score', 'tagline_sentiment', 'profitable', 'budget_adj', 
            'revenue_adj']

In [7]:
X = final_df[features]
y = final_df['total_score']

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [9]:
rf = RandomForestRegressor()

In [10]:
print("Cross Val Score:", cross_val_score(rf, X_train, y_train, cv = 5).mean())

Cross Val Score: 0.9050463294419101


In [11]:
model_params = {}
count = 0

In [19]:
rf_params = {
    'max_depth': [13, 14, 15],
    'max_features': ['auto', 'sqrt'],
    'min_samples_leaf': [0, 1, 2],
    'min_samples_split': [1, 2, 3],
    'n_estimators': [150, 160, 170, 180],
}

gs = GridSearchCV(rf, param_grid=rf_params, cv = 5, n_jobs = 12)

gs.fit(X_train, y_train)

count += 1

gs.best_params_['best_score'] = gs.best_score_

gs.best_params_['training_score'] = gs.score(X_train, y_train)

gs.best_params_['testing_score'] = gs.score(X_test, y_test)

model_params[f'model_{count}'] = gs.best_params_

model_df = pd.DataFrame.from_dict(model_params, orient='index')

model_df

,max_depth,max_features,min_samples_leaf,min_samples_split,n_estimators,best_score,training_score,testing_score
model_1,3,auto,2,2,100,0.764538,0.800653,0.724029
model_2,5,auto,2,2,75,0.860821,0.911801,0.819609
model_3,7,auto,1,3,90,0.895800,0.964926,0.861055
model_4,9,auto,1,2,90,0.904488,0.981748,0.866970
model_5,11,auto,1,3,100,0.905883,0.985402,0.866535
model_6,11,auto,1,2,120,0.907034,0.985698,0.875896
model_7,13,auto,1,2,150,0.906534,0.987343,0.871366
model_8,13,auto,1,3,150,0.906918,0.986472,0.873342


## AdaBoostRegressor

In [20]:
model_params_two = {}
count_two = 0

In [27]:
ada = AdaBoostRegressor(base_estimator=RandomForestRegressor())

ada_params = {
    'base_estimator__max_depth': [11, 12, 13],
    'learning_rate': [.95, 1.],
    'n_estimators': [70, 75, 80],
}

gs_two = GridSearchCV(ada, param_grid=ada_params, cv=3, n_jobs = 12)

gs_two.fit(X_train, y_train)

count_two += 1

gs_two.best_params_['best_score'] = gs_two.best_score_

gs_two.best_params_['training_score'] = gs_two.score(X_train, y_train)

gs_two.best_params_['testing_score'] = gs_two.score(X_test, y_test)

model_params_two[f'model_{count_two}'] = gs_two.best_params_

model_df_two = pd.DataFrame.from_dict(model_params_two, orient='index')

model_df_two

,base_estimator__max_depth,learning_rate,n_estimators,best_score,training_score,testing_score
model_1,3,0.95,20,0.815696,0.861969,0.772930
model_2,5,0.95,30,0.879960,0.953392,0.850893
model_3,7,0.95,40,0.896370,0.983651,0.878017
model_4,9,1.00,45,0.899599,0.992798,0.884188
model_5,11,0.90,55,0.901346,0.995733,0.885692
model_6,12,1.00,70,0.901715,0.996314,0.887713
model_7,13,0.95,70,0.901044,0.996506,0.886476
